# Simple Radar

In [1]:
import collections

import numpy as np
import pamtra2
import pamtra2.libs.refractiveIndex as refractiveIndex
import pytest
import xarray as xr



In [2]:
    @pytest.fixture()

def create_simple_cloud_creator():
    def create_simple_cloud_creator(
            nHeights=1,
            edr=1e-3,
            Ntot=[1],
            scattering=pamtra2.hydrometeors.scattering.Rayleigh,
            relativePermittivity=pamtra2.hydrometeors.relativePermittivity.water_turner_kneifel_cadeddu,
            radarPNoise1000=-30,
            temperature = 250,
            instrument='simple',
            dask=False,
            additionalDims=collections.OrderedDict(),
            size=0.001,
            hydrometeor = 'cloud',
            hydrometeorContent = 0.0001,
            **kwargs
        ):

            additionalDims = additionalDims

            pam2 = pamtra2.pamtra2(
                nLayer=nHeights,
                hydrometeors=['hydrometeor'],
                additionalDims=additionalDims,
                frequencies=[3e9],
            )

            pam2.profile.height[:] = 1000
            pam2.profile.temperature[:] = temperature
            pam2.profile.relativeHumidity[:] = 90
            pam2.profile.pressure[:] = 100000
            pam2.profile.eddyDissipationRate[:] = edr
            pam2.profile.horizontalWind[:] = 10
            pam2.profile.hydrometeorContent[:] = hydrometeorContent

            pam2.profile['heightBinDepth'] = 10
            pam2.addMissingVariables()

            if hydrometeor == 'cloud':
                pam2.describeHydrometeor(
                    pamtra2.hydrometeors.softEllipsoidFixedDensity,
                    name='hydrometeor',  # or None, then str(index)
                    nBins=2,
                    sizeBounds=pamtra2.hydrometeors.size.linspaceBounds,
                    sizeCenter=pamtra2.hydrometeors.size.boundsToMid,
                    sizeBoundsWidth=pamtra2.hydrometeors.size.boundsWidth,
                    sizeDistribution=pamtra2.hydrometeors.sizeDistribution.\
                    monoDisperse,
                    aspectRatio=1.0,
                    mass=pamtra2.hydrometeors.mass.ellipsoid,
                    density=pamtra2.hydrometeors.density.water,
                    crossSectionArea=pamtra2.hydrometeors.crossSectionArea.sphere,
                    relativePermittivity=relativePermittivity,
                    scattering=scattering,
                    fallVelocity=pamtra2.hydrometeors.fallVelocity.\
                    khvorostyanov01_drops,
                    Dmin=size - .5e-10,
                    Dmax=size + .5e-10,
                    Ntot=xr.DataArray(Ntot, coords=[pam2.profile.layer]),
                    checkTemperatureForRelativePermittivity=False,
                    useFuncArgDefaults=False,
                    **kwargs,
                )
            elif hydrometeor == 'snow':
                pam2.describeHydrometeor(
                    pamtra2.hydrometeors.softEllipsoidMassSize,
                    name='hydrometeor',  # or None, then str(index)
                    nBins=10,
                    sizeBounds=pamtra2.hydrometeors.size.linspaceBounds,
                    sizeCenter=pamtra2.hydrometeors.size.boundsToMid,
                    sizeBoundsWidth=pamtra2.hydrometeors.size.boundsWidth,
                    sizeDistribution=pamtra2.hydrometeors.sizeDistribution.exponentialFieldWC,
                    aspectRatio=1.0,
                    mass=pamtra2.hydrometeors.mass.powerLaw,
                    density=pamtra2.hydrometeors.density.softEllipsoid,
                    crossSectionArea=pamtra2.hydrometeors.crossSectionArea.sphere,
                    relativePermittivity=relativePermittivity,
                    scattering=scattering,
                    fallVelocity=pamtra2.hydrometeors.fallVelocity.heymsfield10_particles,
                    Dmin=size,
                    Dmax=size + 0.01,
                    Ntot=xr.DataArray(Ntot, coords=[pam2.profile.layer]),
                    checkTemperatureForRelativePermittivity=False,
                    useFuncArgDefaults=False,
                    massSizeA=0.0121,
                    massSizeB=1.9,
                    minDensity = 100,
                    maxDensity = pamtra2.constants.rhoIce,
                    **kwargs,
                )



            pam2.profile['pathIntegratedAtenuattion'] = xr.zeros_like(
                pam2.hydrometeors.hydrometeor.profile.backscatterCrossSection.isel(
                    sizeBin=0))

            if dask:
                pam2.profile = pam2.profile.chunk({'layer': 1, 'frequency': 1})

            if instrument == 'simple':
                results = pam2.addInstrument(
                    pamtra2.instruments.radar.simpleRadar,
                    name='simple',
                    frequencies=3e9,
                )
            elif instrument == 'spectral':
                results = pam2.addInstrument(
                    pamtra2.instruments.radar.dopplerRadarPamtra,
                    name='spectral',
                    frequencies=3e9,
                    momentsNPeaks=1,
                    seed=11,
                    radarAliasingNyquistInterv=0,
                    radarPNoise1000=radarPNoise1000
                )

            if dask:
                results.results.load()

            return results
    #     return simple_cloud_creator


In [3]:
# def test_relativePermitivity():
turner_kneifel_cadeddu = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=273.15
).results.radarReflectivity.values
ellison = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=273.15,
    relativePermittivity=pamtra2.hydrometeors.relativePermittivity.water_ellison
).results.radarReflectivity.values
ice_warren_brandt_2008 = create_simple_cloud_creator(
    nHeights=1,
    Ntot = [1],
    temperature=260.15,
    relativePermittivity=pamtra2.hydrometeors.relativePermittivity.ice_warren_brandt_2008
).results.radarReflectivity.values
ice_matzler_2006 = create_simple_cloud_creator(
    nHeights=1,
    Ntot = [1],
    temperature=260.15,
    relativePermittivity=pamtra2.hydrometeors.relativePermittivity.ice_matzler_2006
).results.radarReflectivity.values
iwabuchi_yang_2011 = create_simple_cloud_creator(
    nHeights=1,
    Ntot = [1],
    temperature=[260.15],
    relativePermittivity=pamtra2.hydrometeors.relativePermittivity.ice_iwabuchi_yang_2011
).results.radarReflectivity.values

assert np.allclose(turner_kneifel_cadeddu, ellison, rtol=1e-01, atol=1e-01)
assert np.allclose(ice_warren_brandt_2008, ice_matzler_2006, rtol=1e-01, atol=1e-01)
assert np.allclose(iwabuchi_yang_2011, ice_matzler_2006, rtol=1e-01, atol=1e-01)


#     print (turner_kneifel_cadeddu- ellison)

sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x10e3520d0>
callable
crossSectionArea <function sphere at 0x10e3527b8>
callable
sizeDistribution <function monoDisperse at 0x10e34bd90>
callable
relativePermittivity <function turner_kneifel_cadeddu at 0x11b457d90>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function khvorostyanov01_drops at 0x10e352d08>
callable
nBins 2
not callable 2
Dmin 0.00099999995
not callable 0.00099999995
Dmax 0.00100000005
not callable 0.00100000005
Ntot <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
not callable <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
checkTemperatureForRelativePermittivity False
not callable Fals

/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:287: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:
/Volumes/User/mmaahn/anaconda/envs/python3/

sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x10e3520d0>
callable
crossSectionArea <function sphere at 0x10e3527b8>
callable
sizeDistribution <function monoDisperse at 0x10e34bd90>
callable
relativePermittivity <function ellison at 0x11b457e18>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function khvorostyanov01_drops at 0x10e352d08>
callable
nBins 2
not callable 2
Dmin 0.00099999995
not callable 0.00099999995
Dmax 0.00100000005
not callable 0.00100000005
Ntot <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
not callable <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
checkTemperatureForRelativePermittivity False
not callable False


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:287: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:
/Volumes/User/mmaahn/anaconda/envs/python3/

sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x10e3520d0>
callable
crossSectionArea <function sphere at 0x10e3527b8>
callable
sizeDistribution <function monoDisperse at 0x10e34bd90>
callable
relativePermittivity <function warren_brandt_2008 at 0x11b4572f0>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function khvorostyanov01_drops at 0x10e352d08>
callable
nBins 2
not callable 2
Dmin 0.00099999995
not callable 0.00099999995
Dmax 0.00100000005
not callable 0.00100000005
Ntot <xarray.DataArray (layer: 1)>
array([1])
Coordinates:
  * layer    (layer) int64 0
not callable <xarray.DataArray (layer: 1)>
array([1])
Coordinates:
  * layer    (layer) int64 0
checkTemperatureForRelativePermittivity False
not callable False
sizeBounds <

/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:287: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:
/Volumes/User/mmaahn/anaconda/envs/python3/

scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function khvorostyanov01_drops at 0x10e352d08>
callable
nBins 2
not callable 2
Dmin 0.00099999995
not callable 0.00099999995
Dmax 0.00100000005
not callable 0.00100000005
Ntot <xarray.DataArray (layer: 1)>
array([1])
Coordinates:
  * layer    (layer) int64 0
not callable <xarray.DataArray (layer: 1)>
array([1])
Coordinates:
  * layer    (layer) int64 0
checkTemperatureForRelativePermittivity False
not callable False
sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRatio 1.0
not callable 1.0
density 1000.0
not callable 1000.0
mass <function ellipsoid at 0x10e3520d0>
callable
crossSectionArea <function sphere at 0x10e3527b8>
callable
sizeDistribution <function monoDisperse at 0x10e34bd90>
callable
relativePermittivity <function ice_iwabuchi_yang_2011 at 0x10e352e18>
callable
scatteri

/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:287: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:
/Volumes/User/mmaahn/anaconda/envs/python3/

In [4]:


mixing_maxwell_garnett = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=273.15,
    hydrometeor='snow',
    relativePermittivity = pamtra2.hydrometeors.relativePermittivity.mixing_maxwell_garnett,
    relativePermittivityIce = pamtra2.hydrometeors.relativePermittivity.ice_iwabuchi_yang_2011,
    
).results.radarReflectivity.values

mixing_bruggeman = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=273.15,
    hydrometeor='snow',
    relativePermittivity = pamtra2.hydrometeors.relativePermittivity.mixing_bruggeman,
    relativePermittivityIce = pamtra2.hydrometeors.relativePermittivity.ice_iwabuchi_yang_2011,
    
).results.radarReflectivity.values

mixing_sihvola = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=273.15,
    hydrometeor='snow',
    relativePermittivity = pamtra2.hydrometeors.relativePermittivity.mixing_sihvola,
    relativePermittivityIce = pamtra2.hydrometeors.relativePermittivity.ice_iwabuchi_yang_2011,
    
).results.radarReflectivity.values






/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/sizeDistribution.py:329: UserWarning: Truncation effect on the PSD are not considered. I.e., typically mass is lost!
  warnings.warn('Truncation effect on the PSD are not considered. '


sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRatio 1.0
not callable 1.0
mass <function powerLaw at 0x10e352048>
callable
density <function softEllipsoid at 0x10e352950>
callable
crossSectionArea <function sphere at 0x10e3527b8>
callable
relativePermittivityIce <function ice_iwabuchi_yang_2011 at 0x10e352e18>
callable
sizeDistribution <function exponentialFieldWC at 0x10e352400>
callable
relativePermittivity <function mixing_maxwell_garnett at 0x10e352ea0>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function heymsfield10_particles at 0x10e352c80>
callable
nBins 10
not callable 10
Dmin 0.001
not callable 0.001
Dmax 0.011
not callable 0.011
Ntot <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
not callable <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (

/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['Ntot', 'checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:287: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:
/Volumes/User/mmaahn/anaconda/envs/

 <function ice_iwabuchi_yang_2011 at 0x10e352e18>
callable
sizeDistribution <function exponentialFieldWC at 0x10e352400>
callable
relativePermittivity <function mixing_bruggeman at 0x10e352f28>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function heymsfield10_particles at 0x10e352c80>
callable
nBins 10
not callable 10
Dmin 0.001
not callable 0.001
Dmax 0.011
not callable 0.011
Ntot <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
not callable <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
checkTemperatureForRelativePermittivity False
not callable False
massSizeA 0.0121
not callable 0.0121
massSizeB 1.9
not callable 1.9
minDensity 100
not callable 100
maxDensity 917.0
not callable 917.0
sizeBounds <function linspaceBounds at 0x10e34bb70>
callable
sizeCenter <function boundsToMid at 0x10e34bc80>
callable
sizeBoundsWidth <function boundsWidth at 0x10e34bd08>
callable
aspectRat

/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/sizeDistribution.py:329: UserWarning: Truncation effect on the PSD are not considered. I.e., typically mass is lost!
  warnings.warn('Truncation effect on the PSD are not considered. '
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:259: UserWarning: The following kwargs were NOT used: ['Ntot', 'checkTemperatureForRelativePermittivity']
  '%s' % self._keysToBeUsed)
/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/

relativePermittivity <function mixing_sihvola at 0x10e356048>
callable
scattering <function Rayleigh at 0x10e352bf8>
callable
fallVelocity <function heymsfield10_particles at 0x10e352c80>
callable
nBins 10
not callable 10
Dmin 0.001
not callable 0.001
Dmax 0.011
not callable 0.011
Ntot <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
not callable <xarray.DataArray (layer: 2)>
array([1, 2])
Coordinates:
  * layer    (layer) int64 0 1
checkTemperatureForRelativePermittivity False
not callable False
massSizeA 0.0121
not callable 0.0121
massSizeB 1.9
not callable 1.9
minDensity 100
not callable 100
maxDensity 917.0
not callable 917.0


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2-0.1-py3.6-macosx-10.9-x86_64.egg/pamtra2/hydrometeors/core.py:291: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if key in self.profile[key]:


In [5]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [6]:
mixing_maxwell_garnett, mixing_bruggeman#, mixing_sihvola

(array([[47.47471638, 47.47471638]]), array([[47.47471638, 47.47471638]]))

In [7]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [8]:
pamtra2.hydrometeors.relativePermittivity.mixing_maxwell_garnett?



In [9]:
np.complex

complex

In [10]:
xr.broadcast([temperature, frequency])

NameError: name 'temperature' is not defined

In [ ]:
turner_kneifel_cadeddu

In [ ]:
dat = create_simple_cloud_creator(
    nHeights=2,
    Ntot = [1,2],
    temperature=260.15,
    relativePermittivity=pamtra2.hydrometeors.relativePermittivity.ice_iwabuchi_yang_2011
)

In [ ]:
%debug

In [ ]:
xr.broadcast?

In [ ]:
sum?

In [ ]:
np.sum(1)